In [83]:
from infrastructure.database.mongo_client import MongoDBClient
from infrastructure.repository import YoutubeContentRepository
from use_case import YouTubeScriptRefinement
from domain.youtube_timeline_summary import YoutubeTimelineSummary
from domain.youtube_timeline_section import YoutubeTimelineSection
from domain.youtube_content import YouTubeContent


import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

# MongoDB 클라이언트 초기화
client = MongoDBClient(uri=os.environ['MONGO_CONNECTION_STRING'])
client.connect()

# 저장소 초기화
content_repo: YoutubeContentRepository = YoutubeContentRepository(client)

# LLM
openai_llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY)


# Content
contents = content_repo.find_all()
content: YouTubeContent = contents[1]

Connected to MongoDB


# 아웃 파서

In [84]:
from typing import List
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser

# ✅ (핵심 키포인트 모델)
class YoutubeKeyPointModel(BaseModel):
    term: str = Field(..., description="핵심 용어")
    description: str = Field(..., description="핵심 용어에 대한 설명")

# ✅ (핵심 키포인트 컬렉션 모델)
class YoutubeKeyPointCollectionModel(BaseModel):
    key_points: List[YoutubeKeyPointModel] = Field(..., description="영상에서 추출된 핵심 키포인트 리스트")

# ✅ 아웃파서 정의
key_point_parser = PydanticOutputParser(pydantic_object=YoutubeKeyPointCollectionModel)

# 프롬프트 템플릿

In [85]:
from langchain.prompts import PromptTemplate

key_point_prompt_template = PromptTemplate(
    partial_variables={'output_format': key_point_parser.get_format_instructions()},
    input_variables=["title", "descrption", "summary"],
    template="""
다음은 유튜브 영상의 제목, 설명 타임라인 요약 문서입니다. 이 내용을 분석하여 핵심 용어(KeyPoint)와 설명을 생성하세요.
설명을 생성할 때 영상 위주가 아닌 최대한 전문적이고 구체적으로 추가 설명해주세요. 저는 영상 내용 뿐 아니라 그 용어에 대한 내용을 알고 싶습니다.
중요하지 않더라도 알면 좋을 추가 핵심 용어도 더해주세요. 

[제목]:
{title}

[설명]:
{description}

[타임라인 요약]:
{summary}


[가이드]:
- 필드는 `term`과 `description`을 포함하며, 결과는 여러 개의 키포인트 리스트로 구성됩니다.
- 반드시 아래 JSON 형식에 맞춰 반환하세요.
- 다른 텍스트는 포함하지 마세요.

[반환 형식]:
{output_format}
"""
)

# LLM

In [86]:
from langchain_openai import ChatOpenAI

openai_llm = ChatOpenAI(model="gpt-4o-mini", api_key=os.environ['OPENAI_API_KEY'])

In [87]:
script_refine_chain = key_point_prompt_template | openai_llm | key_point_parser


summary: YoutubeTimelineSummary = content.timeline_summary

summary_texts = f"{summary.text}\n"

for section in summary.sections:
    summary_texts = summary_texts + '\n'
    for text in section.texts:
        summary_texts = summary_texts + f'- {text}\n'


print(key_point_prompt_template)

input_variables=['description', 'summary', 'title'] input_types={} partial_variables={'output_format': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"$defs": {"YoutubeKeyPointModel": {"properties": {"term": {"description": "핵심 용어", "title": "Term", "type": "string"}, "description": {"description": "핵심 용어에 대한 설명", "title": "Description", "type": "string"}}, "required": ["term", "description"], "title": "YoutubeKeyPointModel", "type": "object"}}, "properties": {"key_points": {"description": "영상에서 추출된 핵심 키포인트 리스트", "items": {"$ref": "#/$defs/YoutubeKeyPointModel"}, "title": "Key Poi

In [88]:
input_data = {
        "title": content.title,
        "description": content.description,
        "summary": summary_texts
    }

response: YoutubeKeyPointCollectionModel = script_refine_chain.invoke(input_data)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [89]:
for key_point in response.key_points:
    print(f'{key_point.term}: {key_point.description}')

C# 스크립트: Unity에서 게임 로직을 구현하기 위해 사용하는 프로그래밍 언어. C# 스크립트는 게임 오브젝트의 동작, 상호작용, 행동을 정의하는 데 핵심적인 역할을 한다.
입력 받기: 사용자로부터 입력을 받아 게임 내에서의 행동을 제어하는 방법. Unity에서는 input.GetAxis() 함수를 활용하여 사용자 입력을 감지하며, 'Horizontal'과 'Vertical' 축에 따라 방향키 입력을 처리할 수 있다.
물리 이동: 게임 오브젝트의 이동 방식을 물리 법칙에 따라 구현하는 방법. Rigidbody2D 컴포넌트를 사용하여 힘, 속도, 위치를 조절할 수 있으며, 물리 엔진에 기반해 보다 자연스러운 움직임을 구현할 수 있다.
Rigidbody2D: Unity의 2D 물리 엔진에서 게임 오브젝트에 물리적인 특성을 추가하기 위해 사용되는 컴포넌트. 물리적 상호작용을 통해 충돌, 중력 그리고 속도와 관련된 행동을 구현할 수 있다.
FixedUpdate: Unity에서 물리 연산을 처리할 때 사용되는 메소드. 매 프레임이 아닌 일정한 timestep으로 호출되므로, 물리조건의 일관성을 유지하기 위해 Rigidbody와 함께 주로 사용된다.
Normalize: 벡터의 방향성을 유지하면서 벡터의 크기를 1로 맞추는 과정. 플레이어의 이동 방향을 정규화하면, 이동 속도를 균일하게 유지하면서 방향만을 조정할 수 있다.
AddForce: Rigidbody에 힘을 가하여 물리적인 움직임을 주는 메소드. 힘을 주어 오브젝트를 자연스럽게 이동시키는 데 사용되며, 물리 엔진에 의한 시뮬레이션이 실행된다.
Velocity: Rigidbody의 속도를 직접적으로 설정하는 속성. Velocity를 통해 오브젝트의 이동 속도를 정밀하게 조정할 수 있으며, 즉각적인 반응을 필요로 하는 상황에서 유용하다.
MovePosition: Rigidbody의 위치를 직접적으로 변경하는 방법. 간단한 경로 이동을 구현할 때 사용되며, 물리 상호작용을 무시하고 기본적인 이동을 수행할 수 있다.
